In [6]:
import os
import json
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv

load_dotenv()  # .env 파일 불러오기
OPENAI_KEY = os.getenv("OPENAI_KEY")

# 파일 및 디렉토리 설정
date_list_file = "date_list.txt"  # 날짜 목록 파일
data_dir = "summary"  # JSON 파일이 저장된 디렉토리
documents = []

try:
    # 날짜 목록 읽기
    with open(date_list_file, "r", encoding="utf-8") as date_file:
        dates = [line.strip() for line in date_file if line.strip()]  # 빈 줄 제거

    # 날짜별 파일 처리
    for date_raw in dates:
        file_name = f"summary_{date_raw}.json"
        file_path = os.path.join(data_dir, file_name)

        if os.path.exists(file_path):  # 파일 존재 여부 확인
            # 날짜 변환: "250111" -> "2025-01-11"
            date = f"20{date_raw[:2]}-{date_raw[2:4]}-{date_raw[4:]}"

            # JSON 파일 로드
            with open(file_path, "r", encoding="utf-8") as f:
                summary_data = json.load(f)

                # 요약 내용을 문서로 추가
                if "summary" in summary_data:
                    documents.append({
                        "content": summary_data["summary"],
                        "metadata": {"date": date}
                    })
        else:
            print(f"파일 '{file_name}'을(를) 찾을 수 없습니다.")

    print("데이터 통합 완료.")
    print(f"총 {len(documents)}개의 문서가 통합되었습니다.")

    # 통합된 문서 출력 (예시: 처음 5개 문서 출력)
    for doc in documents[:5]:
        print(doc)

    # Step 2: 임베딩 및 벡터 저장소 생성
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)  # OpenAI Embeddings 사용

    # documents 리스트를 Document 객체로 변환
    documents_fixed = [
        Document(page_content=doc["content"], metadata=doc["metadata"])
        for doc in documents
    ]

    # Chroma DB를 영구 저장하기 위한 디렉토리 설정
    persist_directory = "./chroma_db"  # 데이터베이스가 저장될 디렉토리

    # 변환된 리스트를 사용하여 벡터 저장소 생성
    vectorstore = Chroma.from_documents(
        documents_fixed,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    # 벡터 저장소를 디스크에 저장
    vectorstore.persist()

    print("벡터 저장소 생성 및 영구 저장 완료!")

except FileNotFoundError as e:
    print(f"오류: {str(e)}")
except Exception as e:
    print(f"오류 발생: {str(e)}")

데이터 통합 완료.
총 3개의 문서가 통합되었습니다.
{'content': '사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.', 'metadata': {'date': '2025-01-11'}}
{'content': '사용자는 기분이 좋다고 말하며, 딸기가 들어간 크림 모카번을 먹었다. 그는 커피를 마시면 잠을 잘 못 자기 때문에 커피와 함께 먹지 않았다. 그는 또한 소금빵을 좋아하며, 몇 주 전에 마지막으로 먹은 것 같다. 그의 다음 목표는 학교 아래 브레덴코에서 빵을 먹는 것이다. 케이크도 좋아하며, 최근에 생크림 케이크를 먹었다고 했다.', 'metadata': {'date': '2025-01-12'}}
{'content': "오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보고 싶은 음식으로는 떡볶이와 짬뽕을 생각하고 있다. 이번 주말에는 마라탕을 먹을 계획이며, 그 외에 교회를 가는 것도 계획에 있다. 교회에서는 예배드리는 것을 가장 좋아한다. 예배 후에는 친구들과 카페를 가거나, 집에서 쉬곤 한다. 카페에서는 주로 바스트 치즈 케이크와 딸기 라떼를 마신다.", 'metadata': {'date': '2025-01-13'}}
벡터 저장소 생성 및 영구 저장 완료!


C:\Users\dasolkim7\AppData\Local\Temp\ipykernel_23612\2980102256.py:70: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [7]:
# 예시 쿼리
query = "소금빵 먹은 얘기"
results = vectorstore.similarity_search(query, k=3)  # k=3: 상위 3개 결과

print("\n[유사도 검색 결과]")
for i, doc in enumerate(results):
    print(f"결과 {i+1}:")
    print(f"내용: {doc.page_content}")
    print(f"메타데이터: {doc.metadata}")
    print("-----------------------------")



[유사도 검색 결과]
결과 1:
내용: 사용자는 기분이 좋다고 말하며, 딸기가 들어간 크림 모카번을 먹었다. 그는 커피를 마시면 잠을 잘 못 자기 때문에 커피와 함께 먹지 않았다. 그는 또한 소금빵을 좋아하며, 몇 주 전에 마지막으로 먹은 것 같다. 그의 다음 목표는 학교 아래 브레덴코에서 빵을 먹는 것이다. 케이크도 좋아하며, 최근에 생크림 케이크를 먹었다고 했다.
메타데이터: {'date': '2025-01-12'}
-----------------------------
결과 2:
내용: 오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보고 싶은 음식으로는 떡볶이와 짬뽕을 생각하고 있다. 이번 주말에는 마라탕을 먹을 계획이며, 그 외에 교회를 가는 것도 계획에 있다. 교회에서는 예배드리는 것을 가장 좋아한다. 예배 후에는 친구들과 카페를 가거나, 집에서 쉬곤 한다. 카페에서는 주로 바스트 치즈 케이크와 딸기 라떼를 마신다.
메타데이터: {'date': '2025-01-13'}
-----------------------------
결과 3:
내용: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
메타데이터: {'date': '2025-01-11'}
-----------------------------


In [8]:
# Chroma 벡터 스토어 객체가 vectorstore 변수에 있다고 가정
collection = vectorstore._collection

# include 파라미터에서 "ids"는 빼고, documents / embeddings / metadatas만 가져옴
stored_data = collection.get(include=["documents", "embeddings", "metadatas"])

print("=== 저장된 임베딩 정보 ===")

# documents, embeddings, metadatas는 리스트 형태이므로 각각 꺼내서 확인 가능
for i, (doc, embedding, metadata) in enumerate(zip(
    stored_data["documents"], 
    stored_data["embeddings"], 
    stored_data["metadatas"]
)):
    print(f"문서 번호: {i+1}")
    print(f"문서 내용: {doc}")
    print(f"메타데이터: {metadata}")    
    print(f"임베딩 벡터 길이: {len(embedding)}")
    # 임베딩 일부만 출력
    print(f"임베딩 앞부분: {embedding[:10]} ...\n")


=== 저장된 임베딩 정보 ===
문서 번호: 1
문서 내용: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
메타데이터: {'date': '2025-01-11'}
임베딩 벡터 길이: 1536
임베딩 앞부분: [-0.00323614 -0.00233002  0.01288558 -0.03337406 -0.0058267   0.01123392
 -0.01393426 -0.01222361 -0.01021146 -0.00163528] ...

문서 번호: 2
문서 내용: 사용자는 기분이 좋다고 말하며, 딸기가 들어간 크림 모카번을 먹었다. 그는 커피를 마시면 잠을 잘 못 자기 때문에 커피와 함께 먹지 않았다. 그는 또한 소금빵을 좋아하며, 몇 주 전에 마지막으로 먹은 것 같다. 그의 다음 목표는 학교 아래 브레덴코에서 빵을 먹는 것이다. 케이크도 좋아하며, 최근에 생크림 케이크를 먹었다고 했다.
메타데이터: {'date': '2025-01-12'}
임베딩 벡터 길이: 1536
임베딩 앞부분: [-0.00054753 -0.01527661  0.03992392 -0.03609164 -0.00910822  0.00443271
 -0.01584095 -0.0200407  -0.00458692  0.00748081] ...

문서 번호: 3
문서 내용: 오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보고 싶은 음식으로는 떡볶이와 짬뽕을 생각하고 있다. 이번 주말에는 마라탕을 먹을 계획이며, 그 외에 교회를 가는 것도 계획에 있다. 교회에서는 예배드리는 것을 가장 좋아한다. 예배 후에는 친구들과 카페를 가거나,